# Fetch download links for a Dataset's available assets

The script in this notebook retrieves download links for a Dataset's assets.

Fetching a Dataset assets requires only the Collection id (and, for specification purposes, the Dataset dataset_id), and not an API key/access token.

### Import dependencies

In [ ]:
library("readr")
library("httr")
library("stringr")
library("rjson")

#### <font color='#bc00b0'>Please fill in the required values:</font>

<font color='#bc00b0'>(Required) Enter the id of the Collection that contains the Dataset for which you want to fetch download links</font>

_The Collection id can be found by looking at the url path in the address bar 
when viewing your Collection in the UI of the Data Portal website:_ `collections/{collection_id}`.

In [ ]:
collection_id <- "01234567-89ab-cdef-0123-456789abcdef"

<font color='#bc00b0'>(Required) Enter the `dataset_id` of the Dataset whose assets you want to retrieve.</font>

_The `dataset_id` can be found using the_ `/collections/{collection_id}` _endpoint and filtering for this attribute on the Dataset of interest._

In [1]:
dataset_id <- "the_tag"  # Or use the id: "01234567-89ab-cdef-0123-456789abcdef"

### Specify domain (and API url)

In [ ]:
domain_name <- "cellxgene.cziscience.com"
site_url <- str_interp("https://${domain_name}")
api_url_base <- str_interp("https://api.${domain_name}")

### Formulate request and fetch a Dataset's assets

In [ ]:
assets_path <- str_interp("/curation/v1/collections/${collection_id}/datasets/assets")
query_params <- list("dataset_id"=dataset_id)
url <- str_interp("${api_url_base}${assets_path}")
res <- GET(
    url=url, query=query_params, add_headers(`Content-Type`="application/json")
)
stop_for_status(res)
print(content(res))